<a href="https://colab.research.google.com/github/tmyeomans/GEOG_484_W2025/blob/main/Group_supervised_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the Earth Engine Library and authenticate with your password
import ee
ee.Authenticate()

# Initialize the library
ee.Initialize(project= 'ee-tmyeomans')

In [2]:
import geemap

Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305

https://developers.google.com/earth-engine/datasets/catalog/sentinel

In [3]:
# Import Sentinel-2 data
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

In [4]:
true_colour = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],}

In [5]:
s2_map = geemap.Map()
s2_map.set_center(-114.07,51.05, 11)
s2_map.add_layer(s2.mean(), true_colour, 'True Colour')
s2_map

Map(center=[51.05, -114.07], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [6]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [7]:
s2_cf = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(mask_s2_clouds)

In [8]:
s2_cf_map = geemap.Map()
s2_cf_map.set_center(-114.07,51.05, 11)
s2_cf_map.add_layer(s2_cf.mean(), true_colour, 'True Colour')
s2_cf_map

Map(center=[51.05, -114.07], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [21]:
s2_cf_s2024 = s2_cf.filterBounds(aoi).filterDate('2024-07-01', '2024-08-30')

In [47]:
aoi_coords = [
    [-114.3, 50.7],  # Bottom-left corner
    [-114.3, 51.2],  # Top-left corner
    [-113.4, 51.2],  # Top-right corner
    [-113.4, 50.7]   # Bottom-right corner
]

# Create a polygon geometry for the AOI
aoi = ee.Geometry.Polygon(aoi_coords)

In [48]:
s2_aoi = s2_cf.filterBounds(aoi)

s2_aoi_map = geemap.Map()
s2_aoi_map.set_center(-114.07,51.05, 11)
s2_aoi_map.add_layer(s2_aoi, true_colour, 'True Colour')
s2_aoi_map

Map(center=[51.05, -114.07], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [49]:
s2_med = s2_aoi.median().clip(aoi)

In [50]:
s2_cf_s2024_map = geemap.Map()
s2_cf_s2024_map.set_center(-114.07,51.05, 10)
s2_cf_s2024_map.add_layer(s2_med, true_colour, 'True Colour')
s2_cf_s2024_map

Map(center=[51.05, -114.07], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…